<a href="https://colab.research.google.com/github/DaniilGlubshevAndr/LaboratoryWork3/blob/main/Filter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
import numpy as np
from timeit import default_timer as timer
import pycuda.autoinit
from pycuda.driver import In, Out, Context
from pycuda.compiler import SourceModule
from PIL import Image

BLOCK_SIZE = 32
BLOCK = (BLOCK_SIZE, BLOCK_SIZE, 1)
FILTER_SIZE = 5
ARRAY_SIZE = FILTER_SIZE ** 2
DISPLACEMENT = FILTER_SIZE // 2
FILE_NAMES = ["512.bmp", "1024.bmp"]

#ядро
kernel = SourceModule(
    """
    __global__ void median_filter(unsigned char* pixels, unsigned char* filtered, int* size){
        const int blockSize = %(BLOCK_SIZE)s;
        const int arraySize = %(ARRAY_SIZE)s;
        const int filterSize = %(FILTER_SIZE)s;
        const int offset = %(DISPLACEMENT)s;
        int width = size[0];
        int j = blockIdx.x * blockDim.x + threadIdx.x;
	      int i = blockIdx.y * blockDim.y + threadIdx.y; 
	      int x, y, index;
        __shared__ int local[blockSize][blockSize];
        int arr[arraySize];
        local[threadIdx.y][threadIdx.x] = pixels[i * width + j];
        __syncthreads ();
        for (int k = 0; k < filterSize; k++){
            x = max(0, min(threadIdx.y + k - offset, blockSize - 1));
            for (int l = 0; l < filterSize; l++){
                index = k * filterSize + l;
                y = max(0, min(threadIdx.x + l - offset, blockSize - 1));
                arr[index] = local[x][y];
            }
        }
        __syncthreads ();
        for (int k = 0; k < arraySize; k++){
            for (int l = k + 1; l < arraySize; l++){
                if (arr[k] > arr[l]){
                    unsigned char temp = arr[k];
                    arr[k] = arr[l];
                    arr[l] = temp;
                }
            }
        }
        filtered[i * width + j] = arr[int(arraySize / 2)];
    }
    """ % {
        'BLOCK_SIZE': BLOCK_SIZE,
        'ARRAY_SIZE': ARRAY_SIZE,
        'DISPLACEMENT': DISPLACEMENT,
        'FILTER_SIZE': FILTER_SIZE
    }
)

median_filter = kernel.get_function("median_filter")

#функция для загрузки изображения
def open_image(filename: str):
    image = Image.open(filename)
    pix = image.load()
    width = image.size[0]
    height = image.size[1]
    pixels = np.zeros((width, height), dtype=np.uint8)
    for i in range(height):
        for j in range(width):
            pixels[i, j] = pix[j, i]
    return pixels, width, height

#применение медианного фильтра на CPU
def cpu_filter(pixels, width, height):
    filtered = np.zeros_like(pixels)
    mediana = ARRAY_SIZE // 2
    for i in range(height):
        for j in range(width):
            arr = np.zeros(ARRAY_SIZE)
            for k in range(FILTER_SIZE):
                x = max(0, min(i + k - DISPLACEMENT, height - 1))
                index = k * FILTER_SIZE
                for l in range(FILTER_SIZE):
                    y = max(0, min(j + l - DISPLACEMENT, width - 1))
                    arr[index + l] = pixels[x, y]
            arr.sort()
            filtered[i, j] = arr[mediana]
    return filtered

#применение медианного фильтра на GPU
def gpu_filter(pixels, width, height):
    size = np.array([width, height])
    filtered = np.zeros_like(pixels)
    #размер сетки
    grid_dim = (width // BLOCK_SIZE, height // BLOCK_SIZE) 
    #запуск ядра
    median_filter(In(pixels), Out(filtered), In(size), block=BLOCK, grid=grid_dim)
    Context.synchronize()
    return filtered

#сохраняем изображение в формате bmp
def save_image(filtered, filename):
    new_image = Image.fromarray(filtered.astype('uint8'), mode='L')
    new_image.save(filename, format="BMP")

#измеряем время на CPU
def time_cpu(pixels, width, height, save):
    start = timer()
    #запускаем функцию
    filtered = cpu_filter(pixels, width, height)
    cpu_time = timer() - start
    if save:
        save_image(filtered, "cpu" + filename)
    return cpu_time * 1000

#измеряем время на GPU
def time_gpu(pixels, width, height, save):
    start = timer()
    #запускаем функцию
    filtered = gpu_filter(pixels, width, height)
    gpu_time = timer() - start
    if save:
        save_image(filtered, "gpu" + filename)
    return gpu_time * 1000

CPU_TEST_ROUND = 12
GPU_TEST_ROUND = 12

for filename in FILE_NAMES:
    pixels, width, height = open_image(filename)
    time_cpu(pixels, width, height, True)
    time_gpu(pixels, width, height, True)
    cpu_time = 0
    gpu_time = 0
    for i in range(CPU_TEST_ROUND):
        cpu_time += time_cpu(pixels, width, height, False)

    for i in range(GPU_TEST_ROUND):
        gpu_time += time_gpu(pixels, width, height, False)

    cpu_time /= CPU_TEST_ROUND
    gpu_time /= GPU_TEST_ROUND
    #вывод времени и ускорения для каждого файла
    print("Имя файла: {:8s} -  на CPU {:5.5f}, на GPU {:2.5f}, ускорение {:5.5f}".format(filename, cpu_time, gpu_time, cpu_time / gpu_time))

Имя файла: 512.bmp  -  на CPU 4907.35116, на GPU 2.51630, ускорение 1950.22564
Имя файла: 1024.bmp -  на CPU 19760.58671, на GPU 8.44777, ускорение 2339.14856


# Новый раздел

# Новый раздел